# Mega Experiment: Cross‐Validation, Model Comparison and Results Visualization

In this notebook we perform a cross-validation experiment using several forecaster models available in APDTFlow. We:
- Clean and load the time series dataset.
- Create sliding-window datasets.
- Define a rolling-window cross-validation splitting strategy.
- Train multiple models (APDTFlow, TransformerForecaster, TCNForecaster, EnsembleForecaster) on each fold.
- Collect evaluation metrics.
- Save the results to a CSV summary.
- Plot the forecasts (sample) as well as a bar plot to compare average performance.


In [1]:
import os
import sys
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
import seaborn as sns

os.chdir("C:/Users/yotam/code_projects/APDTFlow")
project_root = os.path.abspath(os.path.join(os.getcwd(), "..", ".."))
if project_root not in sys.path:
    sys.path.append(project_root)
print("Project root added to sys.path:", project_root)
print("CUDA available:", torch.cuda.is_available())
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

from torch.utils.data import DataLoader, Subset
from apdtflow.data import TimeSeriesWindowDataset
from apdtflow.models.apdtflow import APDTFlow
try:
    from apdtflow.models.transformer_forecaster import TransformerForecaster
    from apdtflow.models.tcn_forecaster import TCNForecaster
    from apdtflow.models.ensemble_forecaster import EnsembleForecaster
except ImportError:
    print("Warning: One or more alternative forecaster modules are not available.")

Project root added to sys.path: C:\Users\yotam
CUDA available: False


In [2]:
original_csv = "C:/Users/yotam/code_projects/APDTFlow/dataset_examples/daily-minimum-temperatures-in-me.csv"
clean_csv = "C:/Users/yotam/code_projects/APDTFlow/dataset_examples/daily-minimum-temperatures-in-me_clean.csv"
df = pd.read_csv(original_csv)
df["Daily minimum temperatures"] = pd.to_numeric(df["Daily minimum temperatures"], errors='coerce')
df = df.dropna(subset=["Daily minimum temperatures"])
df["Daily minimum temperatures"] = df["Daily minimum temperatures"].astype(np.float32)
df["Date"] = pd.to_datetime(df["Date"])
df.sort_values("Date", inplace=True)
df.to_csv(clean_csv, index=False)
print("Clean CSV saved to:", clean_csv)

Clean CSV saved to: C:/Users/yotam/code_projects/APDTFlow/dataset_examples/daily-minimum-temperatures-in-me_clean.csv


In [3]:
def compute_norm_params(csv_file, value_col):
    df = pd.read_csv(csv_file)
    df[value_col] = df[value_col].astype(np.float32)
    values = df[value_col].values
    return values.mean(), values.std()

mean_val, std_val = compute_norm_params(clean_csv, "Daily minimum temperatures")

def normalize_tensor(x):
    return (x - mean_val) / std_val

def denormalize_tensor(x):
    return x * std_val + mean_val


In [4]:
def run_model_forward(model, x_batch, t_span, device):
    model_name = model.__class__.__name__
    if model_name == 'APDTFlow':
        return model(x_batch, t_span)
    elif model_name == 'TransformerForecaster':
        if x_batch.dim() == 2:
            x_batch = x_batch.unsqueeze(-1) 
        elif x_batch.dim() == 3 and x_batch.size(1) == 1:
            x_batch = x_batch.transpose(1, 2) 
        return model(x_batch)
    elif model_name == 'TCNForecaster':
        if x_batch.dim() == 2:
            x_batch = x_batch.unsqueeze(1) 
        return model(x_batch)
    elif model_name == 'EnsembleForecaster':
        return model.predict(x_batch, None, device)
    else:
        return model(x_batch, t_span)


In [5]:
def time_series_splits(dataset, train_size, step_size, max_splits=3):
    """
    Yields up to max_splits train and validation indices for a rolling window split.
    """
    n_samples = len(dataset)
    count = 0
    for start in range(0, n_samples - train_size, step_size):
        if count >= max_splits:
            break
        train_indices = list(range(start, start + train_size))
        val_indices = list(range(start + train_size, min(start + train_size + step_size, n_samples)))
        yield train_indices, val_indices
        count += 1


def save_forecast_plot(x_batch, y_batch, preds, T_in, T_out, save_path, title="Forecast"):
    sample_idx = 0
    if x_batch.dim() == 3:
        x_sample = x_batch[sample_idx, 0, :].cpu().numpy()
    else:
        x_sample = x_batch[sample_idx].cpu().numpy()
    y_sample = y_batch[sample_idx].squeeze().cpu().numpy()
    if isinstance(preds, tuple):
        preds = preds[0]
    if preds.dim() == 3:
        pred_sample = preds[sample_idx, :, 0].cpu().numpy()
    else:
        pred_sample = preds[sample_idx].cpu().numpy()
    
    x_sample_denorm = denormalize_tensor(torch.tensor(x_sample)).numpy()
    y_sample_denorm = denormalize_tensor(torch.tensor(y_sample)).numpy()
    pred_sample_denorm = denormalize_tensor(torch.tensor(pred_sample)).numpy()
    
    input_timesteps = np.arange(T_in)
    future_timesteps = np.arange(T_in, T_in+T_out)
    
    plt.figure(figsize=(10,6))
    plt.plot(input_timesteps, x_sample_denorm, label="Input Sequence", marker="o")
    plt.plot(future_timesteps, y_sample_denorm, label="True Future", marker="o", linestyle="--")
    plt.plot(future_timesteps, pred_sample_denorm, label="Predicted Future", marker="o", linestyle=":")
    plt.xlabel("Time Step")
    plt.ylabel("Daily Minimum Temperature")
    plt.title(title)
    plt.legend()
    plt.grid(True)
    plt.savefig(save_path)
    plt.close()
    print("Saved forecast plot to:", save_path)

def evaluate_model(model, data_loader, device):
    model.eval()
    total_loss = 0.0
    total_samples = 0
    with torch.no_grad():
        for x_batch, y_batch in data_loader:
            x_batch = x_batch.to(device)
            y_batch = y_batch.to(device)
            if model.__class__.__name__ == 'APDTFlow' and x_batch.dim() == 4 and x_batch.size(1)==1:
                x_batch = x_batch.squeeze(1)
            batch_size = x_batch.size(0)
            if model.__class__.__name__ == 'APDTFlow':
                T_in_current = x_batch.size(-1)
            else:
                if x_batch.dim() == 2:
                    T_in_current = x_batch.size(1)
                elif x_batch.dim() == 3:
                    T_in_current = x_batch.size(-1)
            t_span = torch.linspace(0,1,steps=T_in_current, device=device)
            output = run_model_forward(model, x_batch, t_span, device)
            if isinstance(output, tuple):
                preds = output[0]
            else:
                preds = output
            mse = (preds - y_batch.transpose(1,2)) ** 2
            loss = torch.mean(mse)
            total_loss += loss.item() * batch_size
            total_samples += batch_size
    return total_loss / total_samples

def train_on_split(model, train_loader, val_loader, num_epochs, learning_rate, device):
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    for epoch in range(num_epochs):
        model.train()
        epoch_loss = 0.0
        for x_batch, y_batch in train_loader:
            x_batch = x_batch.to(device)
            y_batch = y_batch.to(device)
            
            if model.__class__.__name__ == 'APDTFlow' and x_batch.dim()==4 and x_batch.size(1)==1:
                x_batch = x_batch.squeeze(1)
            
            batch_size = x_batch.size(0)
            if model.__class__.__name__ == 'APDTFlow':
                T_in_current = x_batch.size(-1)
            else:
                if x_batch.dim() == 2:
                    T_in_current = x_batch.size(1)
                elif x_batch.dim() == 3:
                    T_in_current = x_batch.size(-1)
            t_span = torch.linspace(0, 1, steps=T_in_current, device=device)
            optimizer.zero_grad()
            output = run_model_forward(model, x_batch, t_span, device)
            
            if model.__class__.__name__ == 'APDTFlow':
                preds, pred_logvars = output
                mse = (preds - y_batch.transpose(1,2)) ** 2
                loss = torch.mean(0.5 * (mse / (pred_logvars.exp() + 1e-6)) + 0.5 * pred_logvars)
            else:
                if isinstance(output, tuple):
                    preds = output[0]
                else:
                    preds = output
                mse = (preds - y_batch.transpose(1,2)) ** 2
                loss = torch.mean(mse)
            
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item() * batch_size
        avg_loss = epoch_loss / len(train_loader.dataset)
        print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {avg_loss:.4f}")
    
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for x_batch, y_batch in val_loader:
            x_batch = x_batch.to(device)
            y_batch = y_batch.to(device)
            if model.__class__.__name__ == 'APDTFlow' and x_batch.dim()==4 and x_batch.size(1)==1:
                x_batch = x_batch.squeeze(1)
            batch_size = x_batch.size(0)
            if model.__class__.__name__ == 'APDTFlow':
                T_in_current = x_batch.size(-1)
            else:
                if x_batch.dim() == 2:
                    T_in_current = x_batch.size(1)
                elif x_batch.dim() == 3:
                    T_in_current = x_batch.size(-1)
            t_span = torch.linspace(0, 1, steps=T_in_current, device=device)
            output = run_model_forward(model, x_batch, t_span, device)
            
            if model.__class__.__name__ == 'APDTFlow':
                preds, pred_logvars = output
                mse = (preds - y_batch.transpose(1,2)) ** 2
                loss = torch.mean(0.5 * (mse / (pred_logvars.exp() + 1e-6)) + 0.5 * pred_logvars)
            else:
                if isinstance(output, tuple):
                    preds = output[0]
                else:
                    preds = output
                mse = (preds - y_batch.transpose(1,2)) ** 2
                loss = torch.mean(mse)
            val_loss += loss.item() * batch_size
    avg_val_loss = val_loss / len(val_loader.dataset)
    print(f"Validation Loss: {avg_val_loss:.4f}")
    return avg_val_loss


In [6]:
forecast_horizons = [7, 14, 30]
T_in = 30          
train_size = 400  
step_size = 50  
num_epochs = 35
learning_rate = 0.0007
batch_size = 16

apdtflow_params = {
    "num_scales": 3,         # you might experiment with 4 scales too
    "input_channels": 1,
    "filter_size": 7,         # increased from 5 to 7
    "hidden_dim": 32,         # increased from 16 to 32
    "output_dim": 1,
}

# For TransformerForecaster, increase the model dimension and layers:
transformer_params = {
    "input_dim": 1,
    "model_dim": 32,        # increased from 16 to 32
    "num_layers": 2,        # increased from 1 to 2
    "nhead": 4,
    "forecast_horizon": None
}

# For TCNForecaster, try more channels and a larger kernel:
tcn_params = {
    "input_channels": 1,
    "num_channels": [32, 32],  # increased from [16,16]
    "kernel_size": 7,          # increased from 5 to 7
    "forecast_horizon": None
}


In [7]:
plots_dir = "C:/Users/yotam/code_projects/APDTFlow/experiments/results_plots"
os.makedirs(plots_dir, exist_ok=True)
results_dir = os.path.join(project_root, "experiments")
os.makedirs(results_dir, exist_ok=True)
results_csv_path = os.path.join(results_dir, "results_experiment.csv")


In [8]:
results = []
max_cv_plots = 3

for T_out in forecast_horizons:
    print("\n==========================")
    print(f"Forecast Horizon (T_out): {T_out}")
    print("==========================\n")
    dataset = TimeSeriesWindowDataset(
        csv_file=clean_csv,
        date_col="Date",
        value_col="Daily minimum temperatures",
        T_in=T_in,
        T_out=T_out,
        transform=normalize_tensor
    )
    print("Dataset loaded. Total samples:", len(dataset))
    losses_apdt = []
    losses_trans = []
    losses_tcn = []
    losses_ens = []
    
    cv_index = 0
    for train_idx, val_idx in time_series_splits(dataset, train_size, step_size, max_splits=3):
        cv_index += 1
        print(f"\n--- CV Split {cv_index} ---")
        train_subset = Subset(dataset, train_idx)
        val_subset = Subset(dataset, val_idx)
        train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=False)
        val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False)
        model_apdt = APDTFlow(
            forecast_horizon=T_out,
            **apdtflow_params
        ).to(device)
        try:
            model_trans = TransformerForecaster(
                input_dim=transformer_params["input_dim"],
                model_dim=transformer_params["model_dim"],
                num_layers=transformer_params["num_layers"],
                nhead=transformer_params["nhead"],
                forecast_horizon=T_out
            ).to(device)
        except Exception as e:
            print("TransformerForecaster error:", e)
            model_trans = None
        try:
            model_tcn = TCNForecaster(
                input_channels=tcn_params["input_channels"],
                num_channels=tcn_params["num_channels"],
                kernel_size=tcn_params["kernel_size"],
                forecast_horizon=T_out
            ).to(device)
        except Exception as e:
            print("TCNForecaster error:", e)
            model_tcn = None
        
        ensemble_models = []
        if model_apdt is not None:
            ensemble_models.append(model_apdt)
        if model_trans is not None:
            ensemble_models.append(model_trans)
        if model_tcn is not None:
            ensemble_models.append(model_tcn)
        try:
            model_ens = EnsembleForecaster(models=ensemble_models).to(device)
        except Exception as e:
            print("EnsembleForecaster error:", e)
            model_ens = None
        
        print("\nTraining APDTFlow model...")
        loss_apdt = train_on_split(model_apdt, train_loader, val_loader, num_epochs, learning_rate, device)
        losses_apdt.append(loss_apdt)
        
        if model_trans is not None:
            print("\nTraining TransformerForecaster model...")
            loss_trans = train_on_split(model_trans, train_loader, val_loader, num_epochs, learning_rate, device)
            losses_trans.append(loss_trans)
        else:
            losses_trans.append(np.nan)
            
        if model_tcn is not None:
            print("\nTraining TCNForecaster model...")
            loss_tcn = train_on_split(model_tcn, train_loader, val_loader, num_epochs, learning_rate, device)
            losses_tcn.append(loss_tcn)
        else:
            losses_tcn.append(np.nan)
            
        if model_ens is not None:
            loss_ens = evaluate_model(model_ens, val_loader, device)
            losses_ens.append(loss_ens)
            print(f"\nEnsembleForecaster CV Loss: {loss_ens:.4f}")
        else:
            losses_ens.append(np.nan)
        
        if cv_index <= max_cv_plots:
            full_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False)
            x_batch, y_batch = next(iter(full_loader))
            x_batch = x_batch.to(device)
            y_batch = y_batch.to(device)
            if x_batch.dim() == 4 and x_batch.size(1) == 1:
                x_batch = x_batch.squeeze(1)
            _, T_in_current = x_batch.shape[1:]
            t_span = torch.linspace(0, 1, steps=T_in, device=device)
            
            with torch.no_grad():
                preds_apdt, _ = model_apdt(x_batch, t_span)
            save_forecast_plot(x_batch, y_batch, preds_apdt, T_in, T_out,
                               os.path.join(plots_dir, f"APDTFlow_Forecast_Horizon_{T_out}_CV{cv_index}.png"),
                               title=f"APDTFlow Forecast (Horizon {T_out}, CV{cv_index})")
            
            if model_trans is not None:
                with torch.no_grad():
                    preds_trans = run_model_forward(model_trans, x_batch, t_span, device)
                save_forecast_plot(x_batch, y_batch, preds_trans, T_in, T_out,
                                   os.path.join(plots_dir, f"Transformer_Forecast_Horizon_{T_out}_CV{cv_index}.png"),
                                   title=f"Transformer Forecast (Horizon {T_out}, CV{cv_index})")
            
            if model_tcn is not None:
                with torch.no_grad():
                    preds_tcn = run_model_forward(model_tcn, x_batch, t_span, device)
                save_forecast_plot(x_batch, y_batch, preds_tcn, T_in, T_out,
                                   os.path.join(plots_dir, f"TCN_Forecast_Horizon_{T_out}_CV{cv_index}.png"),
                                   title=f"TCN Forecast (Horizon {T_out}, CV{cv_index})")
            
            if model_ens is not None:
                with torch.no_grad():
                    preds_ens, _ = model_ens.predict(x_batch, T_out, device)
                save_forecast_plot(x_batch, y_batch, preds_ens, T_in, T_out,
                                   os.path.join(plots_dir, f"Ensemble_Forecast_Horizon_{T_out}_CV{cv_index}.png"),
                                   title=f"Ensemble Forecast (Horizon {T_out}, CV{cv_index})")
    
    avg_loss_apdt = np.nanmean(losses_apdt)
    avg_loss_trans = np.nanmean(losses_trans)
    avg_loss_tcn = np.nanmean(losses_tcn)
    avg_loss_ens = np.nanmean(losses_ens)
    
    results.append({
        "Forecast_Horizon": T_out,
        "Model": "APDTFlow",
        "Avg_Validation_Loss": avg_loss_apdt
    })
    results.append({
        "Forecast_Horizon": T_out,
        "Model": "TransformerForecaster",
        "Avg_Validation_Loss": avg_loss_trans
    })
    results.append({
        "Forecast_Horizon": T_out,
        "Model": "TCNForecaster",
        "Avg_Validation_Loss": avg_loss_tcn
    })
    results.append({
        "Forecast_Horizon": T_out,
        "Model": "EnsembleForecaster",
        "Avg_Validation_Loss": avg_loss_ens
    })



Forecast Horizon (T_out): 7

Dataset loaded. Total samples: 3611

--- CV Split 1 ---

Training APDTFlow model...


c:\Users\yotam\code_projects\APDTFlow\venv\lib\site-packages\torch\nn\utils\weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


Epoch 1/35, Training Loss: 14.3999
Epoch 2/35, Training Loss: 5.2590
Epoch 3/35, Training Loss: 4.0981
Epoch 4/35, Training Loss: 3.6622
Epoch 5/35, Training Loss: 3.4077
Epoch 6/35, Training Loss: 3.2272
Epoch 7/35, Training Loss: 3.1022
Epoch 8/35, Training Loss: 3.0157
Epoch 9/35, Training Loss: 2.9490
Epoch 10/35, Training Loss: 2.8979
Epoch 11/35, Training Loss: 2.8542
Epoch 12/35, Training Loss: 2.8185
Epoch 13/35, Training Loss: 2.7868
Epoch 14/35, Training Loss: 2.7565
Epoch 15/35, Training Loss: 2.7300
Epoch 16/35, Training Loss: 2.7036
Epoch 17/35, Training Loss: 2.6770
Epoch 18/35, Training Loss: 2.6504
Epoch 19/35, Training Loss: 2.6271
Epoch 20/35, Training Loss: 2.5978
Epoch 21/35, Training Loss: 2.5684
Epoch 22/35, Training Loss: 2.5423
Epoch 23/35, Training Loss: 2.5119
Epoch 24/35, Training Loss: 2.4837
Epoch 25/35, Training Loss: 2.4527
Epoch 26/35, Training Loss: 2.4189
Epoch 27/35, Training Loss: 2.3878
Epoch 28/35, Training Loss: 2.3537
Epoch 29/35, Training Loss: 

c:\Users\yotam\code_projects\APDTFlow\venv\lib\site-packages\torch\nn\utils\weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


Epoch 1/35, Training Loss: 22.0252
Epoch 2/35, Training Loss: 5.4423
Epoch 3/35, Training Loss: 4.3343
Epoch 4/35, Training Loss: 3.9347
Epoch 5/35, Training Loss: 3.6662
Epoch 6/35, Training Loss: 3.4723
Epoch 7/35, Training Loss: 3.3153
Epoch 8/35, Training Loss: 3.1930
Epoch 9/35, Training Loss: 3.0986
Epoch 10/35, Training Loss: 3.0255
Epoch 11/35, Training Loss: 2.9652
Epoch 12/35, Training Loss: 2.9131
Epoch 13/35, Training Loss: 2.8717
Epoch 14/35, Training Loss: 2.8335
Epoch 15/35, Training Loss: 2.8005
Epoch 16/35, Training Loss: 2.7683
Epoch 17/35, Training Loss: 2.7401
Epoch 18/35, Training Loss: 2.7090
Epoch 19/35, Training Loss: 2.6790
Epoch 20/35, Training Loss: 2.6470
Epoch 21/35, Training Loss: 2.6144
Epoch 22/35, Training Loss: 2.5831
Epoch 23/35, Training Loss: 2.5487
Epoch 24/35, Training Loss: 2.5118
Epoch 25/35, Training Loss: 2.4753
Epoch 26/35, Training Loss: 2.4318
Epoch 27/35, Training Loss: 2.3916
Epoch 28/35, Training Loss: 2.3508
Epoch 29/35, Training Loss: 

c:\Users\yotam\code_projects\APDTFlow\venv\lib\site-packages\torch\nn\utils\weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


Epoch 1/35, Training Loss: 50.7187
Epoch 2/35, Training Loss: 7.1772
Epoch 3/35, Training Loss: 4.5103
Epoch 4/35, Training Loss: 3.8838
Epoch 5/35, Training Loss: 3.6378
Epoch 6/35, Training Loss: 3.5009
Epoch 7/35, Training Loss: 3.4066
Epoch 8/35, Training Loss: 3.3212
Epoch 9/35, Training Loss: 3.2524
Epoch 10/35, Training Loss: 3.1911
Epoch 11/35, Training Loss: 3.1367
Epoch 12/35, Training Loss: 3.0872
Epoch 13/35, Training Loss: 3.0434
Epoch 14/35, Training Loss: 3.0017
Epoch 15/35, Training Loss: 2.9689
Epoch 16/35, Training Loss: 2.9366
Epoch 17/35, Training Loss: 2.9058
Epoch 18/35, Training Loss: 2.8766
Epoch 19/35, Training Loss: 2.8518
Epoch 20/35, Training Loss: 2.8254
Epoch 21/35, Training Loss: 2.7986
Epoch 22/35, Training Loss: 2.7777
Epoch 23/35, Training Loss: 2.7576
Epoch 24/35, Training Loss: 2.7354
Epoch 25/35, Training Loss: 2.7148
Epoch 26/35, Training Loss: 2.6932
Epoch 27/35, Training Loss: 2.6724
Epoch 28/35, Training Loss: 2.6508
Epoch 29/35, Training Loss: 

c:\Users\yotam\code_projects\APDTFlow\venv\lib\site-packages\torch\nn\utils\weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


Epoch 1/35, Training Loss: 10.6770
Epoch 2/35, Training Loss: 4.1604
Epoch 3/35, Training Loss: 3.4915
Epoch 4/35, Training Loss: 3.2187
Epoch 5/35, Training Loss: 3.0554
Epoch 6/35, Training Loss: 2.9505
Epoch 7/35, Training Loss: 2.8829
Epoch 8/35, Training Loss: 2.8309
Epoch 9/35, Training Loss: 2.7900
Epoch 10/35, Training Loss: 2.7502
Epoch 11/35, Training Loss: 2.7143
Epoch 12/35, Training Loss: 2.6748
Epoch 13/35, Training Loss: 2.6364
Epoch 14/35, Training Loss: 2.5963
Epoch 15/35, Training Loss: 2.5543
Epoch 16/35, Training Loss: 2.5111
Epoch 17/35, Training Loss: 2.4650
Epoch 18/35, Training Loss: 2.4179
Epoch 19/35, Training Loss: 2.3724
Epoch 20/35, Training Loss: 2.3229
Epoch 21/35, Training Loss: 2.2771
Epoch 22/35, Training Loss: 2.2308
Epoch 23/35, Training Loss: 2.1876
Epoch 24/35, Training Loss: 2.1469
Epoch 25/35, Training Loss: 2.1091
Epoch 26/35, Training Loss: 2.0757
Epoch 27/35, Training Loss: 2.0493
Epoch 28/35, Training Loss: 2.0251
Epoch 29/35, Training Loss: 

In [9]:
results_df = pd.DataFrame(results)
print("Experiment Results:")
print(results_df)

results_csv_path = os.path.join(results_dir, "results_experiment.csv")
results_df.to_csv(results_csv_path, index=False)
print("Results table saved to:", results_csv_path)


Experiment Results:
    Forecast_Horizon                  Model  Avg_Validation_Loss
0                  7               APDTFlow             1.985979
1                  7  TransformerForecaster            12.408953
2                  7          TCNForecaster            13.574383
3                  7     EnsembleForecaster            12.521067
4                 14               APDTFlow             2.158489
5                 14  TransformerForecaster            12.617352
6                 14          TCNForecaster            13.084925
7                 14     EnsembleForecaster            11.993816
8                 30               APDTFlow             2.002482
9                 30  TransformerForecaster            14.257534
10                30          TCNForecaster            16.954437
11                30     EnsembleForecaster            13.411902
Results table saved to: C:\Users\yotam\experiments\results_experiment.csv


In [10]:
sns.set(style="whitegrid")
plt.figure(figsize=(10,6))
ax = sns.barplot(x="Forecast_Horizon", y="Avg_Validation_Loss", hue="Model", data=results_df)
plt.title("Average Validation Loss by Forecast Horizon and Model")
plt.xlabel("Forecast Horizon")
plt.ylabel("Avg Validation Loss")
plt.legend(title="Model")
bar_plot_path = os.path.join(plots_dir, "Validation_Loss_Comparison.png")
plt.savefig(bar_plot_path)
plt.close()
print("Validation loss comparison plot saved to:", bar_plot_path)

plt.figure(figsize=(10,6))
for model_name in results_df["Model"].unique():
    model_data = results_df[results_df["Model"] == model_name]
    plt.plot(model_data["Forecast_Horizon"], model_data["Avg_Validation_Loss"], marker="o", label=model_name)
plt.xlabel("Forecast Horizon")
plt.ylabel("Avg Validation Loss")
plt.title("Model Performance vs. Forecast Horizon")
plt.legend()
plt.grid(True)
line_plot_path = os.path.join(plots_dir, "Performance_vs_Horizon.png")
plt.savefig(line_plot_path)
plt.close()
print("Performance vs. Horizon plot saved to:", line_plot_path)


Validation loss comparison plot saved to: C:/Users/yotam/code_projects/APDTFlow/experiments/results_plots\Validation_Loss_Comparison.png
Performance vs. Horizon plot saved to: C:/Users/yotam/code_projects/APDTFlow/experiments/results_plots\Performance_vs_Horizon.png
